In [ ]:
!pip install -q --upgrade keras-nlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 33.8 MB/s eta 0:00:00


In [ ]:
!pip install -q --upgrade keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.0.5 which is incompatible.


In [ ]:
import keras
print(keras.__version__)

2.15.0


In [ ]:
import tensorflow as tf
import numpy as np
import keras
import keras_nlp
import tensorflow_datasets as tfds

Using TensorFlow backend


## Setup

Dataset used is SNLI - Stanford Natural Language Inference.
Load using Tensorflow datasets.
Contains over 550k samples in total. We'll be using only 20% of training data

A little about the dataset:
contains 3 part to each sample: hypothesis, premise and label
* Hypothesis: Hypothesis created by the author
* Premise: original caption provided to the author
* Label: Annoted to indicate the similary b/w two sentences which can be one of the three:
  - Contradiction (completely dissimilar sentences)
  - Entailment (similar sentences)
  - Neutral (clear similarity or dissimilarity cannot be established)

In [ ]:
# Note: changed to 1000 samples as model wasn't training
snli_train = tfds.load("snli", split="train[:1000]")
snli_val = tfds.load("snli", split="validation[:500]")
snli_test = tfds.load("snli", split="test[:500]")

# above load the data into tf.data.Dataset objects

In [ ]:
sample = snli_train.batch(4).take(1).get_single_element()
sample

{'hypothesis': <tf.Tensor: shape=(4,), dtype=string, numpy=
 array([b'Washing clothes on a camping trip.', b'A woman walking alone.',
        b'The woman is thinking.',
        b'The woman is practicing to enter a roller derby.'], dtype=object)>,
 'label': <tf.Tensor: shape=(4,), dtype=int64, numpy=array([ 1,  2, -1,  1])>,
 'premise': <tf.Tensor: shape=(4,), dtype=string, numpy=
 array([b'A man washes or dies clothes in a primitive setting.',
        b'A woman is walking her baby with a stroller at the local park.',
        b'This is a pensive women in a grassy setting.',
        b'Woman in red shirt and white cap rollerblading on gray surface.'],
       dtype=object)>}

### Preprocessing

In [ ]:
# there are labels with missing or incorrect labels: label=-1. Remove these
def filter_labels(sample):
  return sample["label"] >= 0

#utility function that splits the example into (x, y) tuple
def split_labels(sample):
  x = (sample["hypothesis"], sample["premise"])
  y = sample["label"]
  return x, y

In [ ]:
train_ds = (
    snli_train.filter(
        filter_labels)
    .map(split_labels, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(16)
)

val_ds = (
    snli_val
    .filter(filter_labels)
    .map(split_labels, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(16)
)

test_ds = (
    snli_test
    .filter(filter_labels)
    .map(split_labels, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(16)
)

### Establish baseline with BERT

In [ ]:
bert_classifier = keras_nlp.models.BertClassifier.from_preset(
    "bert_tiny_en_uncased",
    num_classes=3
)


100%|██████████| 507/507 [00:00<00:00, 331kB/s]
100%|██████████| 16.8M/16.8M [00:01<00:00, 15.2MB/s]
/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/backbone.py:46: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids
/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/backbone.py:46: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids
100%|██████████| 547/547 [00:00<00:00, 532kB/s]
100%|██████████| 226k/226k [00:00<00:00, 697kB/s]


In [ ]:
bert_classifier.summary()

Preprocessor: "bert_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ bert_tokenizer (BertTokenizer)                     │                                              30,522 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "bert_classifier"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                  ┃ Output Shape                           ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)                     │ (None, None)                           │               0 │
├───────────────────────────────────────────────┼────────────────────────────────────────┼─────────────────┤
│ segment_ids (InputLayer)                      │ (None, None)                           │               0 │
├───────────────────────────────────────────────┼────────────────────────────────────────┼─────────────────┤
│ token_ids (InputLayer)                        │ (None, None)                           │               0 │
├───────────────────────────────────────────────┼────────────────────────────────────────┼─────────────────┤
│ bert_backbone (BertBackbone)                  │ {sequence_output: (None, None, 128),   │       4,385,920 │
│                                               │ pooled_output: (None, 128)}            │                 │
├───────────────────────────────────────────────┼────────────────────────────────────────┼─────────────────┤
│ classifier_dropout (Dropout)                  │ (None, 128)                            │               0 │
├───────────────────────────────────────────────┼────────────────────────────────────────┼─────────────────┤
│ logits (Dense)                                │ (None, 3)                              │             387 │
└───────────────────────────────────────────────┴────────────────────────────────────────┴─────────────────┘

 Total params: 4,386,307 (16.73 MB)

 Trainable params: 4,386,307 (16.73 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
bert_classifier.fit(train_ds,
                   validation_data=val_ds,
                   epochs=1)

63/63 [==============================] - 25s 394ms/step - loss: 1.0763 - sparse_categorical_accuracy: 0.4509 - val_loss: 1.0747 - val_sparse_categorical_accuracy: 0.4433


### Evaluate performance on test data

In [ ]:
bert_classifier.evaluate(test_ds)

31/31 [==============================] - 3s 33ms/step - loss: 1.0704 - sparse_categorical_accuracy: 0.4726


[1.0703612565994263, 0.47261664271354675]

### Recompile the model with slightly different learning rate

In [ ]:
bert_classifier = keras_nlp.models.BertClassifier.from_preset(
    "bert_tiny_en_uncased",
    num_classes=3
)

bert_classifier.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=5e-5),
    metrics=['accuracy']
)

bert_classifier.fit(train_ds, validation_data=val_ds, epochs=1)

/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/backbone.py:46: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids
/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/backbone.py:46: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids


63/63 [==============================] - 19s 139ms/step - loss: 1.0912 - accuracy: 0.4018 - val_loss: 1.0755 - val_accuracy: 0.4577


In [ ]:
bert_classifier.evaluate(test_ds)

31/31 [==============================] - 1s 13ms/step - loss: 1.0716 - accuracy: 0.4807


[1.0716173648834229, 0.48073023557662964]

### Recompile with new optimizer AdamW and learning rate schedule

We're using Triangular scheduling which follows the below pattern:

* Increase learning rate gradually upto a certain number of epochs to a maximum value
* Then, drop at the same rate and epochs to a minimum value

This algorithm helps to explore different learning rate across different regions and generalizes better.

In formal terms, linear ramp up for "warmup" steps, then linear decay to zero at "total" steps

In [ ]:
class TriangularSchedule(keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, rate, warmup, total):
    self.rate = rate
    self.warmup = warmup
    self.total = total

  def get_config(self):
    config = {"rate":self.rate, "warmup":self.warmup, "total":self.total}
    return config

  def __call__(self, step): # what happens when the function is called first time
    step = tf.cast(step, dtype=tf.float32)
    rate = tf.cast(self.rate, dtype=tf.float32)
    warmup = tf.cast(self.warmup, dtype=tf.float32)
    total = tf.cast(self.total, dtype=tf.float32)

    warmup_rate = rate * step / warmup
    cooldown_rate = rate * (total - step) / (total - warmup)
    triangular_rate = tf.math.minimum(warmup_rate, cooldown_rate)
    return tf.math.maximum(triangular_rate, 0.0)


In [ ]:
#Example of how rate progresses
total_steps = 20
warm_steps = 10
my_rate = 2
my_step = 1
warm = []
cool = []
triangle = []
for i in range(1, warm_steps+1):
  warm_rate = my_rate * i / warm_steps
  warm.append(warm_rate)
for i in range(warm_steps+1, total_steps+1):
  cool_rate = my_rate * (total_steps - i) / (total_steps - warm_steps)
  cool.append(cool_rate)

In [ ]:
print(warm)
print(cool)

[0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0]
[1.8, 1.6, 1.4, 1.2, 1.0, 0.8, 0.6, 0.4, 0.2, 0.0]


In [ ]:
bert_classifier = keras_nlp.models.BertClassifier.from_preset(
    "bert_tiny_en_uncased",
    num_classes=3
)

epochs = 3
total_steps = sum(1 for _ in train_ds.as_numpy_iterator()) * epochs
warmup_steps = int(total_steps * 0.2)

bert_classifier.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.AdamW(
        TriangularSchedule(1e-4, warmup_steps, total_steps)
    ),
    metrics=['accuracy']
)

bert_classifier.fit(train_ds,
                    validation_data=val_ds,
                    epochs=epochs)

/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/backbone.py:46: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids
/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/backbone.py:46: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids


Epoch 1/3
63/63 [==============================] - 15s 128ms/step - loss: 1.0902 - accuracy: 0.3828 - val_loss: 1.0791 - val_accuracy: 0.4454
Epoch 2/3
63/63 [==============================] - 12s 188ms/step - loss: 1.0635 - accuracy: 0.4579 - val_loss: 1.0643 - val_accuracy: 0.4784
Epoch 3/3
63/63 [==============================] - 4s 68ms/step - loss: 1.0323 - accuracy: 0.4920 - val_loss: 1.0615 - val_accuracy: 0.4536


In [ ]:
tf.math.maximum(2, 4)

<tf.Tensor: shape=(), dtype=int32, numpy=4>

In [ ]:
# evaluate
bert_classifier.evaluate(test_ds)

31/31 [==============================] - 1s 9ms/step - loss: 1.0454 - accuracy: 0.4625


[1.0454307794570923, 0.46247464418411255]

### Save and reload the model

In [ ]:
bert_classifier.save("bert_classifier.keras")
restored_model = keras.models.load_model("bert_classifier.keras")
restored_model.evaluate(test_ds)

/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/task.py:54: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids
/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/task.py:54: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids
/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/backbone.py:46: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids
/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/backbone.py:46: UserWarning: 

31/31 [==============================] - 3s 25ms/step - loss: 1.0454 - sparse_categorical_accuracy: 0.4625


[1.0454306602478027, 0.46247464418411255]

### Performing Inference with the model

In [ ]:
sample = (sample["hypothesis"], sample["premise"])
sample

(<tf.Tensor: shape=(4,), dtype=string, numpy=
 array([b'Washing clothes on a camping trip.', b'A woman walking alone.',
        b'The woman is thinking.',
        b'The woman is practicing to enter a roller derby.'], dtype=object)>,
 <tf.Tensor: shape=(4,), dtype=string, numpy=
 array([b'A man washes or dies clothes in a primitive setting.',
        b'A woman is walking her baby with a stroller at the local park.',
        b'This is a pensive women in a grassy setting.',
        b'Woman in red shirt and white cap rollerblading on gray surface.'],
       dtype=object)>)

In [ ]:
predictions = bert_classifier.predict(sample)

def softmax(x):
  return np.exp(x) / np.exp(x).sum(axis=0)

predictions = softmax(predictions)


Cause: Unable to locate the source code of <function Model.make_predict_function.<locals>.predict_function at 0x79e7a7adbeb0>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: Unable to locate the source code of <function Model.make_predict_function.<locals>.predict_function at 0x79e7a7adbeb0>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1/1 [==============================] - 1s 1s/step


In [ ]:
np.argmax(predictions, axis=1)

array([1, 0, 2, 1])

In [ ]:
np.argmax([[2, 4], [0, 10]], axis=1)

array([1, 1])

### Improvign with RoBERTa

In [ ]:
roberta_classifier = keras_nlp.models.RobertaClassifier.from_preset(
    "roberta_base_en",
    num_classes=3
)

roberta_classifier.fit(train_ds,
                       validation_data=val_ds,
                       epochs=2)
roberta_classifier.evaluate(test_ds)

100%|██████████| 498/498 [00:00<00:00, 1.16MB/s]
100%|██████████| 474M/474M [00:13<00:00, 36.4MB/s]
/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/backbone.py:46: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids
/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/backbone.py:46: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids
100%|██████████| 463/463 [00:00<00:00, 494kB/s]
100%|██████████| 446k/446k [00:00<00:00, 988kB/s]
100%|██████████| 0.99M/0.99M [00:00<00:00, 1.78MB/s]


Epoch 1/2
63/63 [==============================] - 133s 1s/step - loss: 1.1015 - sparse_categorical_accuracy: 0.3417 - val_loss: 1.1000 - val_sparse_categorical_accuracy: 0.3361
Epoch 2/2
31/31 [==============================] - 14s 243ms/step - loss: 0.9149 - sparse_categorical_accuracy: 0.5882


[0.9148991703987122, 0.5882353186607361]

In [ ]:
predictions = roberta_classifier.predict(sample)
print(np.argmax(predictions, axis=1))

Cause: Unable to locate the source code of <function Model.make_predict_function.<locals>.predict_function at 0x79e7aa0fb7f0>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: Unable to locate the source code of <function Model.make_predict_function.<locals>.predict_function at 0x79e7aa0fb7f0>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: Unable to locate the source code of <function Model.make_predict_function.<locals>.step_function.<locals>.run_step at 0x79e7aa3a2e60>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: Unable to locate the source code of <function Model.make_predict_function.<locals>.step_function.<locals>.run_step at 0x79e7aa3a2e60>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1/1 [==============================] - 12s 12s/step
[0 0 0 2]
